In [ ]:
"""
实体机器人控制 - 基于 inf.ipynb 修改
支持两种方式：
1. 直接加载策略（本地推理）
2. 通过 WebSocket 连接远程策略服务器（推荐）
"""
import os
import sys
import time
import math
import tqdm
import pathlib
import dataclasses
import collections
import numpy as np
import torch

from openpi.training import config as _config
from openpi.policies import policy_config
from openpi_client import image_tools

# 导入实体机器人环境
sys.path.append("/home/wyz/openpi/examples/aloha_real")
from examples.aloha_real import real_env as _real_env

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["TORCH_DISABLE_DYNAMO"] = "1"
os.environ["PYTORCH_JIT"] = "0"

# ALOHA 机器人参数
ALOHA_RESIZE_SIZE = 224
ALOHA_ACTION_HORIZON = 25  # 动作块大小


In [ ]:
# ============================================================
# 方式1: 直接加载策略（本地推理）
# ============================================================
def load_policy_local():
    """直接在本地加载策略"""
    print("Loading policy checkpoint ...")
    # 注意：对于 ALOHA 机器人，应该使用 pi05_aloha 配置
    # 如果你有训练好的 ALOHA checkpoint，使用它
    config = _config.get_config("pi05_aloha")  # 或 "pi05_libero" 如果只有这个
    ckpt = "/home/wyz/openpi/checkpoints/pi05_libero_pytorch.pt"  # 修改为你的 checkpoint 路径

    def _no_compile(x, *args, **kwargs):
        return x
    torch.compile = _no_compile

    try:
        torch.jit._state.disable()
    except Exception:
        pass

    policy = policy_config.create_trained_policy(config, ckpt, pytorch_device="cuda:0")
    print("✅ Policy loaded:", type(policy))
    if hasattr(policy, "model"):
        print(" → Using model:", policy.model.__class__.__module__)
    return policy


In [ ]:
# ============================================================
# 方式2: 通过 WebSocket 连接远程策略服务器（推荐）
# ============================================================
from openpi_client import websocket_client_policy

def create_websocket_policy(host="localhost", port=8000):
    """创建 WebSocket 客户端策略"""
    print(f"Connecting to policy server at {host}:{port}...")
    policy = websocket_client_policy.WebsocketClientPolicy(host=host, port=port)
    print("✅ Connected to policy server")
    metadata = policy.get_server_metadata()
    print(f" → Server metadata: {metadata}")
    return policy


In [ ]:
# ============================================================
# 实体机器人环境初始化
# ============================================================
@dataclasses.dataclass
class RobotArgs:
    """机器人运行参数"""
    use_websocket: bool = True  # True: 使用 WebSocket, False: 本地加载策略
    websocket_host: str = "localhost"
    websocket_port: int = 8000
    task_prompt: str = "take the toast out of the toaster"  # 任务描述
    max_steps: int = 1000  # 最大步数
    action_horizon: int = 25  # 动作块大小
    replan_steps: int = 5  # 每 N 步重新规划
    resize_size: int = 224  # 图像尺寸


def init_robot_env(reset_position=None, resize_size=224):
    """初始化实体机器人环境（使用 AlohaRealEnvironment 包装）"""
    from examples.aloha_real import env as _aloha_env
    
    print("Initializing real robot environment...")
    env = _aloha_env.AlohaRealEnvironment(
        reset_position=reset_position,
        render_height=resize_size,
        render_width=resize_size,
    )
    print("✅ Robot environment initialized")
    return env


In [ ]:
# ============================================================
# 主控制循环
# ============================================================
def run_robot_control(args: RobotArgs):
    """运行机器人控制主循环"""
    
    # 1. 初始化策略
    if args.use_websocket:
        print("\n📡 Using WebSocket policy server")
        policy = create_websocket_policy(args.websocket_host, args.websocket_port)
    else:
        print("\n💻 Using local policy")
        policy = load_policy_local()
    
    # 2. 初始化机器人环境（使用 AlohaRealEnvironment，已处理图像格式）
    env = init_robot_env(resize_size=args.resize_size)
    
    # 3. 预热策略（如果是本地策略）
    if not args.use_websocket:
        print("\n🔥 Warming up policy...")
        # ALOHA 策略期望的格式：图像为 [C, H, W]
        dummy_obs = {
            "state": np.zeros(14, dtype=np.float32),  # ALOHA: 14维状态 (7+7)
            "images": {
                "cam_high": np.zeros((3, args.resize_size, args.resize_size), dtype=np.uint8),
                "cam_left_wrist": np.zeros((3, args.resize_size, args.resize_size), dtype=np.uint8),
                "cam_right_wrist": np.zeros((3, args.resize_size, args.resize_size), dtype=np.uint8),
            },
            "prompt": args.task_prompt,
        }
        with torch.inference_mode():
            _ = policy.infer(dummy_obs)
        torch.cuda.synchronize()
        print("✅ Warmup done")
    
    # 4. 重置环境
    print("\n🔄 Resetting robot...")
    env.reset()
    print("✅ Robot reset complete")
    
    # 5. 主控制循环
    print(f"\n🚀 Starting control loop (task: '{args.task_prompt}')")
    print("⚠️  按 Ctrl+C 停止机器人\n")
    
    action_plan = collections.deque()
    step_count = 0
    actions_from_chunk_completed = 0
    
    try:
        while step_count < args.max_steps:
            # 获取当前观测（AlohaRealEnvironment 已处理图像格式为 [C, H, W]）
            obs = env.get_observation()
            
            # 检查必需的相机图像
            images = obs["images"]
            if "cam_high" not in images:
                print("⚠️  Warning: Missing cam_high image")
                time.sleep(0.1)
                continue
            
            # 获取状态（qpos: 14维，包含左右臂各7个关节）
            state = obs["state"]  # shape: (14,)
            
            # 当动作计划用完或需要重新规划时，调用策略
            if len(action_plan) == 0 or actions_from_chunk_completed >= args.replan_steps:
                actions_from_chunk_completed = 0
                
                # 构建观测字典（ALOHA 策略期望的格式）
                # AlohaRealEnvironment 返回的图像已经是 [C, H, W] 格式
                element = {
                    "state": state.astype(np.float32),
                    "images": images,  # 字典，键为相机名称，值已经是 [C, H, W] 格式
                    "prompt": args.task_prompt,
                }
                
                # 调用策略推理
                try:
                    with torch.inference_mode():
                        t0 = time.perf_counter()
                        result = policy.infer(element)
                        actions = result["actions"]  # shape: (action_horizon, action_dim)
                        infer_time = (time.perf_counter() - t0) * 1000
                        
                        if step_count % 10 == 0:
                            print(f"Step {step_count}: Inference time: {infer_time:.2f} ms")
                    
                    # 将动作块添加到计划中
                    action_plan.extend(actions[:args.replan_steps])
                    
                except Exception as e:
                    print(f"❌ Error during inference: {e}")
                    break
            
            # 执行动作
            if len(action_plan) > 0:
                action = action_plan.popleft()
                actions_from_chunk_completed += 1
                
                # ALOHA 动作格式: [left_arm(6), left_gripper(1), right_arm(6), right_gripper(1)]
                # 确保动作维度正确
                if len(action) == 14:
                    env.apply_action({"actions": action.tolist()})
                else:
                    print(f"⚠️  Warning: Action dimension mismatch. Expected 14, got {len(action)}")
                    break
            else:
                print("⚠️  Warning: No actions in plan")
                time.sleep(0.1)
            
            step_count += 1
            
            # 控制频率（约 10Hz）
            time.sleep(0.1)
    
    except KeyboardInterrupt:
        print("\n\n⚠️  Interrupted by user")
    except Exception as e:
        print(f"\n\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        print(f"\n✅ Control loop finished. Total steps: {step_count}")
        print("⚠️  机器人已停止，请手动检查机器人状态")


In [ ]:
# ============================================================
# 运行示例
# ============================================================

# 方式1: 使用 WebSocket 服务器（推荐）
# ⚠️ 注意：当前服务器使用的是 LIBERO 环境
# 如果要控制实体 ALOHA 机器人，需要重新启动服务器：
# 1. 停止当前服务器（Ctrl+C）
# 2. 运行：uv run scripts/serve_policy.py --env ALOHA --default_prompt='take the toast out of the toaster'

# 如果当前服务器是 LIBERO，可以尝试使用（但可能不兼容）
args_websocket = RobotArgs(
    use_websocket=True,
    websocket_host="localhost",  # 如果服务器在其他机器，修改为服务器IP
    websocket_port=8000,
    task_prompt="pick up the cup",  # LIBERO 任务提示（如果使用 LIBERO 服务器）
    max_steps=1000,
    action_horizon=25,
    replan_steps=5,
)

# ⚠️ 重要：在运行之前，请确保：
# 1. ROS 节点已启动：roslaunch aloha ros_nodes.launch
# 2. 策略服务器已启动（当前是 LIBERO，建议切换到 ALOHA）
# 3. 机器人硬件已连接

# 取消注释以运行：
# run_robot_control(args_websocket)


In [ ]:
# 方式2: 本地加载策略
args_local = RobotArgs(
    use_websocket=False,
    task_prompt="take the toast out of the toaster",
    max_steps=1000,
    action_horizon=25,
    replan_steps=5,
)

# 取消注释以运行：
# run_robot_control(args_local)


## 使用说明

### ⚠️ 重要：ROS 环境问题

**当前问题**：你的系统只有 ROS2 (Humble)，但 ALOHA 需要 ROS1 (Noetic)。

**解决方案**：

#### 方案 A：使用 Docker（推荐，最简单）

Docker 会自动处理 ROS 环境，无需手动安装：

```bash
# 1. 设置策略服务器参数
export SERVER_ARGS="--env ALOHA --default_prompt='take the toast out of the toaster'"

# 2. 启动所有服务（包括 ROS 节点和策略服务器）
docker compose -f examples/aloha_real/compose.yml up --build
```

这会自动启动：
- ROS master
- ALOHA ROS 节点
- 策略服务器
- 机器人运行时

#### 方案 B：安装 ROS1 Noetic（如果不用 Docker）

如果你不想用 Docker，需要先安装 ROS1 Noetic：

```bash
# 1. 添加 ROS Noetic 源
sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
sudo apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654

# 2. 安装 ROS Noetic
sudo apt update
sudo apt install ros-noetic-desktop-full

# 3. 初始化 rosdep
sudo rosdep init
rosdep update

# 4. 设置环境变量（添加到 ~/.bashrc）
echo "source /opt/ros/noetic/setup.bash" >> ~/.bashrc
source ~/.bashrc

# 5. 构建 ALOHA 工作空间（需要按照 ALOHA 文档操作）
```

### 前置条件（如果使用 Docker，可跳过）

1. **启动 ROS 节点**（在另一个终端）：
   ```bash
   # 需要先 source ROS 环境
   source /opt/ros/noetic/setup.bash
   roslaunch aloha ros_nodes.launch
   ```

2. **启动策略服务器**（如果使用 WebSocket 方式，在另一个终端）：
   ```bash
   uv run scripts/serve_policy.py --env ALOHA --default_prompt='take the toast out of the toaster'
   ```

### 运行步骤

1. 确保机器人硬件已连接并初始化
2. 运行上面的代码单元初始化环境和策略
3. 取消注释 `run_robot_control()` 调用并运行
4. 观察机器人执行任务
5. 按 Ctrl+C 停止

### 注意事项

- ⚠️ **安全第一**：确保机器人在安全位置，周围无障碍物
- ⚠️ **检查动作范围**：确保动作在机器人工作空间内
- ⚠️ **监控状态**：随时准备按 Ctrl+C 停止
- 📝 **相机配置**：根据你的策略训练时使用的相机名称调整代码中的相机选择
- 📝 **Checkpoint 路径**：如果使用本地策略，确保 checkpoint 路径正确
- 🐳 **推荐使用 Docker**：避免 ROS 环境配置问题


In [4]:
import pyrealsense2 as rs

ctx = rs.context()
devices = ctx.query_devices()

print("Number of devices:", len(devices))
for dev in devices:
    print(
        dev.get_info(rs.camera_info.name),
        dev.get_info(rs.camera_info.serial_number)
    )


Number of devices: 1
Intel RealSense D435 243222070778


In [5]:
import pyrealsense2 as rs
import numpy as np

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

pipeline.start(config)

for i in range(30):
    frames = pipeline.wait_for_frames()
    color = frames.get_color_frame()
    depth = frames.get_depth_frame()
    print(i, color is not None, depth is not None)

pipeline.stop()


0 True True
1 True True
2 True True
3 True True
4 True True
5 True True
6 True True
7 True True
8 True True
9 True True
10 True True
11 True True
12 True True
13 True True
14 True True
15 True True
16 True True
17 True True
18 True True
19 True True
20 True True
21 True True
22 True True
23 True True
24 True True
25 True True
26 True True
27 True True
28 True True
29 True True
